# Test code

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
import pickle
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *
from community_selection.usertools import *

In [3]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 2100 * np.ones(1), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 90 * np.ones(1), #Number of resources in each class
    'Sgen': 0, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 12,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 4, # Number of total transfer, or number of passage
    "n_transfer_selection": 2, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "n_migration": 10**6, # Number of cells to be migrated in the migration perturbation algorithm
    "R_percent": 0.5, # Fracion of new resources to be spiked in to the media in the resource perturbation algorithm
    "selected_function": "f1_additive"
})

# Prepare experiment setup in this universe
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

In [4]:
data_directory = "data/test/"
#list_algorithms = ["Blouin2015_control", "Swenson2000a_control", "Swenson2000b_control", "Wright2019_control"]
#list_algorithms = ["select_top25_bottleneck_10", "select_top25_bottleneck_100", "select_top25_bottleneck_1000", "select_top25_bottleneck_10000", "pool_top25_bottleneck_10", "pool_top25_bottleneck_100", "pool_top25_bottleneck_1000", "pool_top25_bottleneck_10000"]
#list_algorithms = ["ctrl", "resource_old", "resource", "resource_add", "resource_remove", "resource_rescale_add", "resource_rescale_remove", "knock_in_isolates", "knock_out"]
#list_algorithms = ["ctrl", "resource_old", "resource_add", "resource_remove", "resource_rescale_add", "resource_rescale_remove", "knock_in_isolates", "knock_out"]
#list_algorithms = ["resource_add", "resource_old"]
#list_algorithms = ["select_top25_bottleneck_100000", "select_top25_bottleneck_1000000", "pool_top25_bottleneck_100000", "pool_top25_bottleneck_1000000"]
#list_algorithms = ["resource_add", "bottleneck_1000", "bottleneck_10000", "bottleneck_100000"]
#list_algorithms = ["ctrl", "resource_old", "resource_add", "resource_remove", "resource_rescale_add", "resource_rescale_remove", "bottleneck_1000", "bottleneck_10000", "bottleneck_100000"]
#list_algorithms = ["select_top25_bottleneck_10", "select_top25_bottleneck_100", "select_top25_bottleneck_1000", "select_top25_bottleneck_10000", "select_top25_bottleneck_100000", "select_top25_bottleneck_1000000", "pool_top25_bottleneck_10", "pool_top25_bottleneck_100", "pool_top25_bottleneck_1000", "pool_top25_bottleneck_10000", "pool_top25_bottleneck_100000", "pool_top25_bottleneck_1000000"]
#list_algorithms = ["Chang2020a", "Chang2020a_control", "Chang2020b", "Chang2020b_control"]
#list_algorithms = ["bottleneck_1", "bottleneck_10", "bottleneck_100", "resource_add_10", "resource_add_2", "resource_add_1"]
list_algorithms = ["bottleneck_1"]

# Parameters for simulation
params_simulation.update({"selected_function": "f1_additive"}) # selected function

# Make the list of algorithms
algorithms = make_algorithms(params_simulation)

# Simulation
for i in range(len(list_algorithms)):
    simulate_community(
        assumptions = assumptions,
        params = params,
        dynamics = dynamics,
        params_simulation = params_simulation,
        params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
        write_composition = True,
        write_function = True,
        file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
        assembly_type = str(list_algorithms[i]),
    )


Algorithm: bottleneck_1


 transfer community_phenotype selection_algorithm migration_algorithm
        1         f1_additive        no_selection        no_migration
        2         f1_additive          select_top        no_migration
        3         f1_additive        no_selection        no_migration
        4         f1_additive        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm bottleneck_1 finished


In [19]:
import pickle
plate_screen = pickle.load(open(data_directory + "plate_screen.p", "rb"))
np.random.choice(np.where(plate_screen["plate_N"]["W0"] == 0)[0])

399

In [ ]:
a = [[1,2],[2,],4]
